In [1]:
# Read the predefined entities from the dtd file and the content of the xml file
# with open('dblp.xml', 'r') as f:
#     content_lines = f.readlines()
    
# with open('new_head.xml', 'r') as f:
#     head_lines = f.readlines()

In [2]:
# Replace the head of the xml file with the predefined entities
# with open('dblp_new_head.xml', 'w') as f:
#     lines = head_lines + content_lines[2:]
#     for number, line in enumerate(lines):
#         f.write(line)

In [33]:
with open('data/dblp_new_head.xml', 'r') as f:
    for i in range(0,80):
        line = f.readline()
        if i > 65:
            print(line.strip())

<!ENTITY yacute  "&#253;" ><!-- small y, acute accent -->
<!ENTITY thorn   "&#254;" ><!-- small thorn, Icelandic -->
<!ENTITY yuml    "&#255;" ><!-- small y, dieresis or umlaut mark -->
]><dblp>
<article mdate="2020-06-25" key="tr/meltdown/s18" publtype="informal">
<author>Paul Kocher</author>
<author>Daniel Genkin</author>
<author>Daniel Gruss</author>
<author>Werner Haas 0004</author>
<author>Mike Hamburg</author>
<author>Moritz Lipp</author>
<author>Stefan Mangard</author>
<author>Thomas Prescher 0002</author>
<author>Michael Schwarz 0001</author>


# Get All Tag Info

In [22]:
from xml.sax.handler import ContentHandler
from xml.sax import parse
import json
import re
import pprint



class XMLHandler(ContentHandler):
    
    def __init__(self):
        self.article_num = 0
        self.proceedings_num = 0
        self.inproceedings_num = 0

    def startElement(self, name, attrs):
        if name == "article": 
            self.article_num += 1
        elif name == "proceedings": 
            self.proceedings_num += 1
        elif name == "inproceedings": 
            self.inproceedings_num += 1
        # print(f"BEGIN: <{name}>, {attrs.keys()}")

    def endElement(self, name):
        pass
        #print(f"END: </{name}>")

    def characters(self, content):
        pass
#         if content.strip() != "":
#             print("CONTENT:", repr(content))

In [15]:
l = [1,2,2,3]
s = "\n\n\n\n\n\n"
pat = re.compile(r"(\n)*")
if re.fullmatch(pat, s):
    print("Match!")
else:
    print("not")

Match!


In [17]:
d1 = {"name": "hh"}
d2 = d1
d2["name"] = "yy"
d1

{'name': 'yy'}

In [44]:
"""
Data Structure:
{
    "TagName": string,
    "SingleChildList": [string],
    "MultiChildList": [string],
    "Attriset": {string},
    "TagValue": Boolean,
}

"""

class XMLHandler_stru(ContentHandler):

    def __init__(self):
        super().__init__()
#         self.element_stack = []
        self.tag_dic_list = []
        self.tag_stack = []
        

#     @property
#     def current_element(self):
#         return self.element_stack[-1]

    def startElement(self, name, attrs):
        # add current tag to self.tag_stack
        self.tag_stack.append({
            "TagName": name,
            "SingleChildSet": set(),
            "MultiChildSet": set(),
        })
        # add attri names to the tag's AttriList
        new_tag = True
        for dic in self.tag_dic_list:
            if dic["TagName"] == name:
                for a in dict(attrs).keys():
                    if a not in dic["Attriset"]:
                        dic["Attriset"].add(a)
                new_tag = False
                break
        if new_tag:
            self.tag_dic_list.append({
                "TagName": name,
                "ChildSet": set(),
                "MultiChildSet": set(),
                "Attriset": set(dict(attrs).keys()),
                "TagValue": False,
            })
        
        
#         self.element_stack.append({
#             "name": name,
#             "attributes": dict(attrs),
#             "children": [],
#             "value": ""
#         })
    
    def characters(self, content):
#         print("Content:", content)
#         input()
        # if not null, set "TagValue" to True
        pat = re.compile(r"(\n)*")
        if not re.fullmatch(pat, content):
            current_tag = self.tag_stack[-1]["TagName"]
            for dic in self.tag_dic_list:
                if dic["TagName"] == current_tag:
                    dic["TagValue"] = True
        
        
        
        #self.current_element["value"] += content
        
    def endElement(self, name):
        if len(self.tag_stack) > 1:
            parent_dic = self.tag_stack[-2]
            for dic in self.tag_dic_list:
                if dic["TagName"] == parent_dic["TagName"]:
                    parent_final_dic = dic

            # add current tag to its parent tag dic
            if (name not in parent_dic["SingleChildSet"]) and \
            (name not in parent_dic["MultiChildSet"]):
                parent_dic["SingleChildSet"].add(name)
            elif name in parent_dic["SingleChildSet"]:
                parent_dic["MultiChildSet"].add(name)
                parent_dic["SingleChildSet"].remove(name)

            # merge the parent tag child info
            for child in parent_dic["SingleChildSet"]:
                if child not in parent_final_dic["ChildSet"]:
                    parent_final_dic["ChildSet"].add(child)
            for child in parent_dic["MultiChildSet"]:
                if child not in parent_final_dic["ChildSet"]:
                    parent_final_dic["ChildSet"].add(child)
                if child not in parent_final_dic["MultiChildSet"]:
                    parent_final_dic["MultiChildSet"].add(child)
        
        #debug
#         print("\n\n\n")
#         print("Current tag:", name, "\n")
#         print("Final tag info:")
#         pprint.pprint(self.tag_dic_list)
#         print("\nTemp tag info:")
#         pprint.pprint(self.tag_stack)
#         input()
        
        # remove current tag from self.tag_stack
        self.tag_stack.pop(-1)
        
        
#         if name in ["article", "proceedings", "inproceedings"]:
#             print(json.dumps(self.current_element, indent=4))
#             input()
#         clean(self.current_element)
#         if len(self.element_stack) > 1:
#             child = self.element_stack.pop()
#             self.current_element["children"].append(child)

# def clean(element):
#     element["value"] = element["value"].strip()
#     for key in ("attributes", "children", "value"):
#         if not element[key]:
#             del element[key]

In [45]:
import time

t1 = time.time()

handler = XMLHandler_stru()
parse("data/dblp_new_head.xml", handler)

t2 = time.time()

print("Time taken:", t2-t1)
# print(handler.article_num)
# print(handler.proceedings_num)
# print(handler.inproceedings_num)


Time taken: 1577.7637174129486


NameError: name 'name' is not defined

In [46]:

print("Final tag info:")
pprint.pprint(handler.tag_dic_list)
print("\nTemp tag info:")
pprint.pprint(handler.tag_stack)

Final tag info:
[{'Attriset': set(),
  'ChildSet': {'article',
               'book',
               'data',
               'incollection',
               'inproceedings',
               'mastersthesis',
               'phdthesis',
               'proceedings',
               'www'},
  'MultiChildSet': {'article',
                    'book',
                    'data',
                    'incollection',
                    'inproceedings',
                    'mastersthesis',
                    'phdthesis',
                    'proceedings',
                    'www'},
  'TagName': 'dblp',
  'TagValue': True},
 {'Attriset': {'cdate', 'mdate', 'publtype', 'key'},
  'ChildSet': {'author',
               'booktitle',
               'cdrom',
               'cite',
               'crossref',
               'editor',
               'ee',
               'journal',
               'month',
               'note',
               'number',
               'pages',
               'publisher',
    

In [48]:
tag_info_list = handler.tag_dic_list.copy()
for dic in tag_info_list:
    for mc in dic["MultiChildSet"]:
        dic["ChildSet"].remove(mc)
        
print("Final tag info:")
pprint.pprint(tag_info_list)

Final tag info:
[{'Attriset': set(),
  'ChildSet': set(),
  'MultiChildSet': {'article',
                    'book',
                    'data',
                    'incollection',
                    'inproceedings',
                    'mastersthesis',
                    'phdthesis',
                    'proceedings',
                    'www'},
  'TagName': 'dblp',
  'TagValue': True},
 {'Attriset': {'cdate', 'mdate', 'publtype', 'key'},
  'ChildSet': {'booktitle',
               'cdrom',
               'crossref',
               'journal',
               'month',
               'number',
               'pages',
               'publisher',
               'publnr',
               'title',
               'volume',
               'year'},
  'MultiChildSet': {'url', 'editor', 'note', 'ee', 'cite', 'author'},
  'TagName': 'article',
  'TagValue': True},
 {'Attriset': {'bibtex', 'orcid', 'aux'},
  'ChildSet': set(),
  'MultiChildSet': set(),
  'TagName': 'author',
  'TagValue': True},
 {

In [49]:
# save tag info dictionary
import pickle
with open('tag_info_dic.pkl', 'wb') as fp:
    pickle.dump(tag_info_list, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


# Check Abstract Tag Info below

In [3]:
# load tag info from pickle
import pickle

with open('tag_info_dic.pkl', 'rb') as fp:
    tag_info_list = pickle.load(fp)
    print('Tag dictionary list successfully loaded')

Tag dictionary list successfully loaded


In [5]:
import pprint

tag_L1 = ['article',
            'book',
            'data',
            'incollection',
            'inproceedings',
            'mastersthesis',
            'phdthesis',
            'proceedings',
            'www']
for dic in tag_info_list:
    if dic["TagName"] in tag_L1:
       pprint.pprint(dic) 

{'Attriset': {'key', 'mdate', 'publtype', 'cdate'},
 'ChildSet': {'booktitle',
              'cdrom',
              'crossref',
              'journal',
              'month',
              'number',
              'pages',
              'publisher',
              'publnr',
              'title',
              'volume',
              'year'},
 'MultiChildSet': {'editor', 'note', 'author', 'ee', 'cite', 'url'},
 'TagName': 'article',
 'TagValue': True}
{'Attriset': {'key', 'publtype', 'mdate'},
 'ChildSet': {'booktitle',
              'cdrom',
              'crossref',
              'month',
              'note',
              'publisher',
              'series',
              'title',
              'url',
              'volume',
              'year'},
 'MultiChildSet': {'editor', 'pages', 'author', 'ee', 'cite', 'isbn', 'school'},
 'TagName': 'book',
 'TagValue': False}
{'Attriset': {'key', 'publtype', 'mdate'},
 'ChildSet': {'address',
              'booktitle',
              'journal'

In [6]:
# check data structure of a tag
tag_to_check = "author"
for dic in tag_info_list:
    if dic["TagName"] == tag_to_check:
       pprint.pprint(dic) 

{'Attriset': {'bibtex', 'orcid', 'aux'},
 'ChildSet': set(),
 'MultiChildSet': set(),
 'TagName': 'author',
 'TagValue': True}


In [ ]:
"""
Problems to address:
1. what are keys for each table
2. are there any null values for each attribute
"""


In [ ]:

root = handler.current_element
print(json.dumps(root, indent=4))

In [1]:
from bs4 import BeautifulSoup

In [7]:
with open('dblp.xml', 'r') as f:
    data = f.read()

In [ ]:
Bs_data = BeautifulSoup(data, "xml")

In [6]:
# Finding all instances of tag
# `book`
b_book = Bs_data.find_all('author')
 
print(b_book)

[<author>Alejandro P. Buchmann</author>, <author>M. Tamer  zsu</author>, <author>Dimitrios Georgakopoulos</author>]


In [1]:
import xml.etree.ElementTree as ET

In [2]:
tree = ET.parse('./sample.xml')
root = tree.getroot()

In [3]:
print(ET.tostring(tree.getroot()))

b'<dblp>\n    <article mdate="2019-10-25" key="tr/gte/TR-0146-06-91-165" publtype="informal">\n        <author>Alejandro P. Buchmann</author>\n        <author>M. Tamer &#214; zsu</author>\n        <author>Dimitrios Georgakopoulos</author>\n        <title>Towards a Transaction Management System for DOM.</title>\n        <journal>GTE Laboratories Incorporated</journal>\n        <volume>TR-0146-06-91-165</volume>\n        <month>June</month>\n        <year>1991</year>\n        <url>db/journals/gtelab/index.html#TR-0146-06-91-165</url>\n    </article>\n</dblp>'


In [ ]:
tree = ET.parse('./dblp.xml')
root = tree.getroot()

In [ ]:
import lxml.etree as ET

In [ ]:
with open('./dblp.xml') as fobj:
    xml = fobj.read()

In [ ]:
tree = ET.parse(xml)

In [ ]:
root = tree.getroot()